In [ ]:
import os
from dotenv import load_dotenv
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SupabaseVectorStore
from langchain.document_loaders import TextLoader
from langchain.document_loaders import TextLoader

load_dotenv()
print(load_dotenv())

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [5]:
# configure these to fit your needs
exclude_dir = [".git", "node_modules", "public", "assets"]
exclude_files = ["package-lock.json", ".DS_Store"]
exclude_extensions = [
    ".jpg",
    ".jpeg",
    ".png",
    ".gif",
    ".bmp",
    ".tiff",
    ".ico",
    ".svg",
    ".webp",
    ".mp3",
    ".wav",
]

documents = []

for dirpath, dirnames, filenames in os.walk("CGPT_Token"):
    # skip directories in exclude_dir
    dirnames[:] = [d for d in dirnames if d not in exclude_dir]

    for file in filenames:
        print(f"file: {file}")
        _, file_extension = os.path.splitext(file)

        # skip files in exclude_files
        if file not in exclude_files and file_extension not in exclude_extensions:
            file_path = os.path.join(dirpath, file)
            loader = TextLoader(file_path, encoding="ISO-8859-1")
            documents.extend(loader.load())

file: ChainGPT.sol
file: ChainGPT.sol_flattened.sol
file: compiler_config.json
file: readme.md
file: ChainGPT.json
file: ChainGPT_metadata.json
file: 3c8d2d39e478be63f86f987c03ca34a1.json


# split the text 
# create chunks for the vector database
# insert into db 

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

for doc in docs:
    source = doc.metadata["source"]
    cleaned_source = "/".join(source.split("/")[1:])
    doc.page_content = (
        "FILE NAME: "
        + cleaned_source
        + "\n###\n"
        + doc.page_content.replace("\u0000", "")
    )

embeddings = OpenAIEmbeddings()

print(os.environ.get("TABLE_NAME"))

vector_store = SupabaseVectorStore.from_documents(
    docs, embeddings, client=supabase, table_name=os.environ.get("TABLE_NAME")
)

# now we setup to ask the db/table questions